In [193]:
import os, sys
import numpy as np
import pandas as pd
import importlib
import rpy2
from scipy.io import mmread
sys.path.append('/path/to/repo/performance_evaluation/development_code/')
import modeval

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [217]:
importlib.reload(modeval)

<module 'modeval' from '/home/sturmno1/Projects/Codes/repos/ml_tools/modeval.py'>

In [139]:
result_dir = '/db/melloddy/users_workspace/sturmno1/machine_learning_experiments/chembl_model_02172020_trivial_key2/results/'
model_dir = '/db/melloddy/users_workspace/sturmno1/machine_learning_experiments/chembl_model_02172020_trivial_key2/models/'

In [142]:
# 1/ load the performance metrics from the *metrics.csv file in "results" folder
modeval.perf_from_metrics?

Signature:
modeval.perf_from_metrics(
    result_dir,
    tasks_for_eval=None,
    model_name='Y',
    prefix=None,
    n_cv=5,
    verify=False,
    hidden_sizes=None,
    last_dropout=None,
    learning_rate=None,
    learning_steps=None,
    weight_decay=None,
    epochs=None,
    fold_va=None,
    fold_te=None,
    verbose=False,
)
Docstring:
Collects the performance from the results/*-metrics.csv files. This metrics file contain positive/negtive counts.
    Assumes filename have the default naming convention (i.e. --filename was not used). 
#     :param string result_dir: path to the model folder containing the conf.npy files
#     :param np.array (integers like) tasks_for_eval: tasks to consider for evaluation (default=None)
#     :param bool aggrgate: if True, aggregate results over folds (considering all tasks verifying MIN_SAMPLES)
#     :param string model_name: adds a name in a column to resulting dataframe (default=Y)
#     :param string prefix: used as a filter, if specifi

In [155]:
# note task 4 is missing validation metrics because it is not represented in the validation fold
perf_metrics = modeval.perf_from_metrics(result_dir,verbose=True)
perf_metrics

Loaded 1 metrics files


,task,num_pos,num_neg,num_pos_va,num_neg_va,auc_tr,auc_va,auc_pr_tr,auc_pr_va,avg_prec_tr,...,kappa_va,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,fold_test,model
0,0,26,28,10,7,0.994048,0.728571,0.992851,0.823777,0.993056,...,0.328947,400,0.2,1e-05,0.001,10,20,0,None,Y
1,1,25,28,5,11,0.967647,0.781818,0.969932,0.458889,0.970786,...,0.522388,400,0.2,1e-05,0.001,10,20,0,None,Y
2,2,25,27,4,2,0.908571,0.500000,0.901868,0.579167,0.904375,...,0.000000,400,0.2,1e-05,0.001,10,20,0,None,Y
3,3,28,29,10,13,0.868056,0.807692,0.799224,0.813080,0.814172,...,0.530612,400,0.2,1e-05,0.001,10,20,0,None,Y
4,4,29,27,0,0,0.950192,NaN,0.961724,NaN,0.962284,...,NaN,400,0.2,1e-05,0.001,10,20,0,None,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3542,3542,6353,5695,1322,1099,0.998651,0.923334,0.998866,0.939423,0.998866,...,0.685518,400,0.2,1e-05,0.001,10,20,0,None,Y
3543,3543,4038,8010,845,1576,0.998554,0.929600,0.997475,0.898452,0.997475,...,0.682354,400,0.2,1e-05,0.001,10,20,0,None,Y
3544,3544,2308,9740,472,1949,0.998740,0.936893,0.995308,0.834398,0.995309,...,0.646414,400,0.2,1e-05,0.001,10,20,0,None,Y
3545,3545,8736,3312,1756,665,0.997385,0.939993,0.999029,0.974911,0.999029,...,0.728763,400,0.2,1e-05,0.001,10,20,0,None,Y


In [144]:
# 2/ collect the performance scores from the *conf.npy in the "models" folder
modeval.perf_from_conf?

Signature:
modeval.perf_from_conf(
    model_dir,
    tasks_for_eval=None,
    aggregate=False,
    model_name='Y',
)
Docstring:
Collects the performance from thje models/*-conf.npy files. Useful for HP search because it includes HPs details.
#     :param string model_dir: path to the model folder containing the conf.npy files
#     :param np.array (integers like) tasks_for_eval: tasks to consider for evaluation (default=None)
#     :param bool aggrgate: if True, uses the aggregate results (considering all tasks verifying MIN_SAMPLES)
#     :param string model_name: adds a name in a column to resulting dataframe (default=Y)
#     :return dtype: pandas df containing performance summaries from conf file (including HP: epoch, valid fold, hidden sizes, lr steps ...)
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py
Type:      function


In [156]:
# a/ Lets get performance of each individual task. 
# Note task 4 has NaN auc_pr_va and auc_va --> task 4 is not present in validation fold 0
perf_conf = modeval.perf_from_conf(model_dir, aggregate=False)
perf_conf

,task,fold_te,fold_va,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,n_tasks_eval,min_samples,auc_va,auc_pr_va,model
0,0,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.785714,0.838308,Y
1,1,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.672727,0.361825,Y
2,2,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.500000,0.579167,Y
3,3,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.838462,0.829287,Y
4,4,None,0,20,400,0.2,0.0,0.001,10,3547,25,NaN,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3542,3542,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.922612,0.939367,Y
3543,3543,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.926225,0.896534,Y
3544,3544,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.935405,0.830107,Y
3545,3545,None,0,20,400,0.2,0.0,0.001,10,3547,25,0.935401,0.973494,Y


In [157]:
# b/ we can also directly load performance aggregates (averaged over the tasks) from the conf file
modeval.perf_from_conf(model_dir, aggregate=True)

,fold_te,fold_va,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,min_samples,auc_va_mean,auc_pr_va_mean,max_f1_va_mean,kappa_va_mean,train_time_1epochs,model
0,None,0,20,400,0.2,0.0,0.001,10,25,0.770633,0.71674,0.757492,0.303348,12.306314,Y


In [196]:
# aggregate performance over folds
modeval.aggregate_fold_perf?


Signature: modeval.aggregate_fold_perf(metrics_df, min_samples, n_cv=5, verify=True)
Docstring:
HP performance aggregation over folds. 
    From the metrics dataframe yielded by perf_from_metrics(), does the aggregation over the fold (mean, std) results in one perf per fold.
#     :param pandas df metrics_df: metrics dataframe yielded by perf_from_metrics() 
#     :param int min_sample: minimum number of each class (overal) to be considered in mean
#     :param int n_cv: number of folds to look for
#     :param bool verify: checks for missing folds runs in CV and prints a report if missing jobs
#     :return dtype: pandas df containing performance per task aggregated over each fold
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py
Type:      function


In [197]:
modeval.aggregate_fold_perf(perf_metrics, 50)

Fold runs found : ['0']


,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,auc_tr_mean,auc_va_mean,auc_pr_tr_mean,...,auc_tr_stdev,auc_va_stdev,auc_pr_tr_stdev,auc_pr_va_stdev,avg_prec_tr_stdev,avg_prec_va_stdev,max_f1_tr_stdev,max_f1_va_stdev,kappa_tr_stdev,kappa_va_stdev
0,400,0.2,1e-05,0.001,10,20,0,0.975565,0.777175,0.956839,...,0.023477,0.138973,0.069231,0.237505,0.06889,0.23073,0.071605,0.19585,0.1801,0.250562


In [203]:
modeval.aggregate_fold_perf(perf_metrics, 50, n_cv=1)

,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,auc_tr_mean,auc_va_mean,auc_pr_tr_mean,...,auc_tr_stdev,auc_va_stdev,auc_pr_tr_stdev,auc_pr_va_stdev,avg_prec_tr_stdev,avg_prec_va_stdev,max_f1_tr_stdev,max_f1_va_stdev,kappa_tr_stdev,kappa_va_stdev
0,400,0.2,1e-05,0.001,10,20,0,0.975565,0.777175,0.956839,...,0.023477,0.138973,0.069231,0.237505,0.06889,0.23073,0.071605,0.19585,0.1801,0.250562


In [208]:
# aggregate performance over all dataframe
modeval.aggregate_overall?

Signature: modeval.aggregate_overall(metrics_df, min_samples)
Docstring:
HP performance aggregation overall . 
    From the metrics dataframe yielded by perf_from_metrics(), does the aggregation over the CV (mean, std) results in one perf hyperparameter.
#     :param pandas df metrics_df: metrics dataframe yielded by perf_from_metrics() 
#     :param int min_sample: minimum number of each class (overal) to be considered in mean
#     :return dtype: pandas df containing performance per hyperparameter setting
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py
Type:      function


In [209]:
# aggregate performance over folds --> get average per task
modeval.aggregate_task_perf?

Signature: modeval.aggregate_task_perf(metrics_df, min_samples, n_cv=5, verify=True)
Docstring:
HP performance aggregation over tasks. 
    From the metrics dataframe yielded by perf_from_metrics(), does the aggregation over the CV (mean, std) results in one perf per task.
#     :param pandas df metrics_df: metrics dataframe yielded by perf_from_metrics() 
#     :param int min_sample: minimum number of each class (overal) to be considered in mean
#     :param int n_cv: number of folds to look for
#     :param bool verify: checks for missing folds runs in CV and prints a report if missing jobs
#     :return dtype: pandas df containing performance per task aggregated over CV
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py
Type:      function


In [214]:
perf_conf.columns

Index(['task', 'fold_te', 'fold_va', 'hp_epochs', 'hp_hidden_sizes',
       'hp_last_dropout', 'hp_weight_decay', 'hp_learning_rate',
       'hp_learning_steps', 'n_tasks_eval', 'min_samples', 'auc_va',
       'auc_pr_va', 'model'],
      dtype='object')

In [212]:
perf_metrics.columns

Index(['task', 'num_pos', 'num_neg', 'num_pos_va', 'num_neg_va', 'auc_tr',
       'auc_va', 'auc_pr_tr', 'auc_pr_va', 'avg_prec_tr', 'avg_prec_va',
       'max_f1_tr', 'max_f1_va', 'kappa_tr', 'kappa_va', 'hp_hidden_sizes',
       'hp_last_dropout', 'hp_weight_decay', 'hp_learning_rate',
       'hp_learning_steps', 'hp_epochs', 'fold_va', 'fold_test', 'model'],
      dtype='object')

In [215]:
# melt (for ggplot2)
modeval.melt_perf?

Signature: modeval.melt_perf(df_res, perf_metrics=['prauc', 'rocauc'])
Docstring:
Melts (or unpivot) the performance dataframe resuting from perf_from_conf(). 
#     :param pandas df_res: dataframe containing results as provided by perf_from_conf()
#     :param list strings perf_metrics: list of perf metrics to keep (depends on columns in df_res)
#     :return dtype: pandas df containing performance in melted format usefull for R ggplot2
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py
Type:      function


In [220]:
perf_confm = modeval.melt_perf(perf_conf)
perf_confm

,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,fold_va,variable,value
14188,20,400,0.2,0.0,0.001,10,0,auc_va,0.785714
14189,20,400,0.2,0.0,0.001,10,0,auc_va,0.672727
14190,20,400,0.2,0.0,0.001,10,0,auc_va,0.500000
14191,20,400,0.2,0.0,0.001,10,0,auc_va,0.838462
14192,20,400,0.2,0.0,0.001,10,0,auc_va,NaN
...,...,...,...,...,...,...,...,...,...
21277,20,400,0.2,0.0,0.001,10,0,auc_pr_va,0.939367
21278,20,400,0.2,0.0,0.001,10,0,auc_pr_va,0.896534
21279,20,400,0.2,0.0,0.001,10,0,auc_pr_va,0.830107
21280,20,400,0.2,0.0,0.001,10,0,auc_pr_va,0.973494


In [221]:
# Find out the best hyperparameters
modeval.best_hyperparam(perf_confm)

,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,variable,fold_va,value
1,20,400,0.2,0.0,0.001,10,auc_pr_va,0,0.716740
0,20,400,0.2,0.0,0.001,10,auc_va,0,0.770633


In [219]:
# metling can also be done with perf_metrics, but need to specify metrics columns
modeval.melt_perf(perf_metrics, perf_metrics=['auc_pr_va'])

,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,variable,value
28376,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.823777
28377,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.458889
28378,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.579167
28379,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.813080
28380,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,NaN
...,...,...,...,...,...,...,...,...,...
31918,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.939423
31919,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.898452
31920,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.834398
31921,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.974911
